In [1]:
# Modulos básicos
import numpy as np
import time
#from pylab import imshow
import matplotlib.pyplot as plt
from tqdm import tqdm, tnrange, tqdm_notebook
# Modulo para manejo de fecha
from datetime import datetime, timedelta
# Modulos para astrofisica/solar
import astropy
from sunpy.net import vso
import astropy.units as u
from sunpy.map import Map
from astropy.io import fits # to fix headers
# Custom-made methods and classes for fixing headers
from lib.CompatMaps import sinehpc_wcs_frame_mapping

# Obtener los datos
Se usa el cliente VSO de SunPy para obtener los datos automáticamente, solo se tienen que cambiar las fechas correspondientes. Fechas de interés para el proyecto son:
* 2011/02/12 a 2011/02/17
* 2012/01/23 a 2012/01/28
* 2013/03/04 a 2013/03/09
* 2014/09/23 a 2013/09/28
* 2015/09/03 a 2015/09/08
* 2016/03/11 a 2016/03/16

In [ ]:
# defining datetime range and number of samples 
dates = []  # where the dates pairs are going to be stored
date_start = datetime(2012,1,29,0,0,0)
date_end = datetime(2012,1,30,15,0,0)
date_samples = 157  # Number of samples to take between dates
date_delta = (date_end - date_start)/date_samples  # How frequent to take a sample
date_window = timedelta(minutes=1)
temp_date = date_start
while temp_date < date_end:
    dates.append((str(temp_date),str(temp_date+date_window)))
    temp_date += date_delta    

In [ ]:
# definir instrumento
instrument = 'hmi'
# definir rango de longitud de onda (min,max)
#wavelength = 400*u.nm , 700*u.nm

# Query data - Buscar datos en esas fechas
t = 0
for i in dates:
    tstart, tend = i[0], i[1]
    data_client = vso.VSOClient()
    data_query = data_client.query(vso.attrs.Time(tstart, tend), \
                                   vso.attrs.Instrument(instrument), vso.attrs.Physobs("intensity"))
    print("Found ",len(data_query)," records from ", tstart, " to ", tend)
    print("Time range: ", data_query.time_range())
    print("Size in KB: ", data_query.total_size())
    data_dir = '/home/ivan/projects/Physics/solar/solar-physics-ex/rotation/data/{file}.fits'
    results = data_client.get(data_query, path=data_dir)
    if t%2 == 0:
        time.sleep(30)
    t+=1

In [ ]:
# Fallo en el índice 39 => '2012-01-29 09:41:16.433124' esto es para seguir descargando desde ahí
# definir instrumento
instrument = 'hmi'
# definir rango de longitud de onda (min,max)
#wavelength = 400*u.nm , 700*u.nm

# Query data - Buscar datos en esas fechas
t = 0
for i in dates[149:]:
    tstart, tend = i[0], i[1]
    data_client = vso.VSOClient()
    data_query = data_client.query(vso.attrs.Time(tstart, tend), \
                                   vso.attrs.Instrument(instrument), vso.attrs.Physobs("intensity"))
    print("Found ",len(data_query)," records from ", tstart, " to ", tend)
    print("Time range: ", data_query.time_range())
    print("Size in KB: ", data_query.total_size())
    data_dir = '/home/ivan/projects/Physics/solar/solar-physics-ex/rotation/data/{file}.fits'
    results = data_client.get(data_query, path=data_dir)
    print(results)
    if t%2 == 0:
        time.sleep(60)
    t+=1

In [ ]:
dates[149]

In [ ]:
# Guardar resultados (Solo hace falta hacerlo una vez)
data_dir = '/home/ivan/projects/Physics/solar/solar-physics-ex/rotation/data/{file}.fits'
results = data_client.get(data_query, path=data_dir).wait()

# Arreglando headers de archivos FITS descargados
Desafortunadamente para ver los resultados hay muchas imágenes que no tienen los headers correctos, entonces es necesario hacer algo como lo que se muestra en este enlace para corregir estos problemas: http://docs.sunpy.org/en/latest/generated/gallery/gallery/hmi_synoptic_maps.html and specially the updated example in: https://gist.github.com/Cadair/cbc73dc7888b9bae5d06708270aedd68

In [2]:
basepath = !pwd
fits_files = !ls data/*.fits  # Quedan con el path data/*.fits 
np.shape(fits_files)

(145,)

In [ ]:
#from IPython.display import clear_output
# Read header and data from all fits files programatically
#i = 0  # index for files
for file in fits_files:
    fitsfile = fits.open(basepath[0]+'/'+file)
    fitsfile.verify('fix')
    header = fitsfile[1].header
    del fitsfile
    #time.sleep(1)
    #header.values

In [ ]:
# Fixing headers programatically
for file in tqdm_notebook(fits_files):
    print(file)
    fitsfile = fits.open(basepath[0]+'/data/'+file)
    fitsfile.verify('fix')
    header = (fitsfile[0].header)
    if header['CUNIT2'] == 'sin(latitude)' and header['CTYPE1'] == 'CRLN-CEA' and header['CTYPE2'] == 'CRLT-CEA':
        tqdm.write("Wrong header information found in %s. Changing..." % file)
        #print("Wrong header information found in %s. Changing..." % file)
        header['CUNIT2'] = 'deg'
        header['CTYPE1'] == 'CSLN-CEA'
        header['CTYPE2'] == 'CSLT-CEA'
    try:
        if header['HGLN_OBS'] == 'nan':
            del header['HGLN_OBS']
    except KeyError:
        #tdqm.write("Warning: Key not found, letting it pass.")
        print("Warning: Key not found, letting it pass.")
        pass
    if header.get('CD1_2') == None:
        header['CD1_2'] = 0
    if header.get('CD2_1') == None:
        header['CD2_1'] = 0

In [ ]:
astropy.wcs.utils.WCS_FRAME_MAPPINGS.append([sinehpc_wcs_frame_mapping]) # Adding the map to the WCS astropy utils module

# Graficar visualizar resultados de la búsqueda

In [ ]:
# Esto es para ver alguna imagen en específico, con coordenadas y todo
%matplotlib qt5
# Making the map 
fitsfile = fits.open(basepath[0]+'/'+fits_files[144])
fitsfile.verify('fix')
data = fitsfile[1].data
header = fitsfile[1].header
m = Map((data, header))
m.peek()
# Set the colorbar properties.
#m.plot_settings['cmap'] = 'hmimag'
#m.plot_settings['norm'] = plt.Normalize(-1500, 1500)

In [ ]:
# Para dibujar perfiles
plt.plot(data[2048,:])
plt.xlabel('Pixeles')
plt.ylabel('Intensidad')
plt.grid(True)
plt.show()

# Widget to see time evolution

Since we are really dealing with a "big" sequence of images in FITS format it is helpful to have a widget to actually see the sequence and check wether the time range chosen is correct and see what to expect beforehand.

In [ ]:
%matplotlib inline
from IPython.html.widgets import *
number_of_images = len(fits_files)

def image_sequence(image):
    #fitsfile = fits.open(basepath[0]+'/data/'+fits_files[0])
    #print(fitsfile)
    fitsfile = fits.open(basepath[0]+'/'+fits_files[image])
    fitsfile.verify('fix')
    plt.imshow(fitsfile[1].data)
    plt.title(fitsfile.filename())
    plt.show()
    
interact(image_sequence, image=(0,number_of_images-1,1), continuous_update=False)

# Procesamiento de datos

Aplicaré algo de lo que dicen acá http://www.pyimagesearch.com/2016/10/31/detecting-multiple-bright-spots-in-an-image-with-python-and-opencv/ para detectar manchas simultáneamente y hacer una estadística automática y "grande".

Tengo que aplicar lo de esa página pero con oscuros, no con claros. Por ahora se aplica a una imágen en específico, luego hay que aplicarla a todas las imágenes y que lance los datos de velocidad/trayectoria.

## Algoritmo:

### 1) Remover background/borde oscuro, de medidas en casos anteriores sabemos que

3851 px = Diametro horizontal sol

3967 - 117 = 3850 px = Diametro vert. sol

Recordar que X -> vertical y Y -> horizontal, para tratamiento de imágenes.

In [3]:
# Módulos básicos para detectar contornos/manchas
from imutils import contours
from skimage import measure
import skimage
#import numpy as np
import imutils
import cv2

La imagen es de 4096 x 4096 y si asumimos que la imagen está bien centrada en 2048, entonces podemos hacer un círculo de radio $r\le 3850/2 \approx 1920$ pixeles, tal que lo que esté por fuera de este círculo se vuelva "nan" y lo de adentro se quede igual.

Más información en: http://stackoverflow.com/questions/8647024/how-to-apply-a-disc-shaped-mask-to-a-numpy-array

In [4]:
# Borrar background/contorno negro
## se hace una máscara circular con centro = (2048, 2048)
centerx, centery = 2048, 2048
r = 1920  # radius
y,x = np.ogrid[-centerx:4096-centerx, -centery:4096-centery]
mask = x*x + y*y > r*r  # circular mask
#array = np.ones((n, n))
#array[mask] = 255

In [5]:
# FUNCIONES BÁSICAS PARA HACER LO QUE NECESITO (deberían ir en un .py aparte - WIP)
# 
def extract_fits_data(index):
    '''
    Function to extract data from fitsfile given just an index
    '''
    fitsfile = fits.open(basepath[0]+'/'+fits_files[index])
    fitsfile.verify('fix')
    return fitsfile[1].data

In [6]:
# Apply mask
data = extract_fits_data(14)
data[mask] = 100000
# plot data to check background removal
%matplotlib qt5
plt.figure()
plt.imshow(data)
plt.figure()
plt.imshow(extract_fits_data(14))
plt.show()

 [astropy.io.fits.verify]


### 2) Suavizar la imagen aplicando un filtro Gaussiano

In [7]:
# Just blur the image with a gaussian filter
blurred = cv2.GaussianBlur(data, (31, 31), 0)

In [ ]:
# Plot blurred image
%matplotlib qt5
plt.imshow(blurred)
plt.colorbar()
plt.show()

Al parecer usar 31 en el filtro Gaussiano da buenos resultados con esta imagen que tiene algunas manchas MUY pequeñas, de tal forma que puedo descartar todas las cosas que sean más brillantes a 4.5e4 (45000), detectando así solo las dos manchas grandes. Porque no puedo garantizar que las manchas pequeñas siempre sean visibles.

### 3) "Binarizando" con umbral

In [8]:
thresh = cv2.threshold(blurred, 28000, 74078, cv2.THRESH_BINARY)[1]

In [ ]:
# Plot binary image
%matplotlib qt5
plt.imshow(thresh)
plt.colorbar()
plt.show()

In [9]:
# Removing small "blobs" with erode and dilate
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

### 4) Labeling the spots
Using https://en.wikipedia.org/wiki/Connected-component_labeling

In [10]:
labels = measure.label(thresh, neighbors=8, background=74078)
mask = np.zeros(thresh.shape, dtype="uint8")
# loop over the unique components
for label in np.unique(labels):
    # if this is the background label, ignore it
    if label == 0:
        continue
 
    # otherwise, construct the label mask and count the
    # number of pixels 
    labelMask = np.zeros(thresh.shape, dtype="uint8")
    labelMask[labels == label] = 255
    numPixels = cv2.countNonZero(labelMask)
 
    # if the number of pixels in the component is sufficiently
    # large, then add it to our mask of "large blobs"
    if numPixels > 30:
        mask = cv2.add(mask, labelMask)

In [ ]:
# Plot the resulting mask
# Plot binary image
%matplotlib qt5
plt.imshow(mask)
plt.colorbar()
plt.show()

In [11]:
image = data.astype("uint8")
# find the contours in the mask, then sort them from left to right
cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = contours.sort_contours(cnts)[0]
cv2.namedWindow("Image", cv2.WINDOW_NORMAL or cv2.WINDOW_AUTOSIZE)  # To show window that's larger than screen 
# loop over the contours
for (i, c) in enumerate(cnts):
    # draw the bright spot on the image
    (x, y, w, h) = cv2.boundingRect(c)
    ((cX, cY), radius) = cv2.minEnclosingCircle(c)
    cv2.circle(image, (int(cX), int(cY)), int(radius), (0, 0, 255), 3)
    cv2.putText(image, "#{}".format(i + 1), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

# show the output image
cv2.imshow("Image",image)
cv2.waitKey(0)

233

In [ ]:
cv2.WINDO

In [ ]:
A = skimage.util.invert(data)

In [ ]:
m.data = blurred

In [ ]:
np.max(np.nan_to_num(data))

In [ ]:
thresh